In [ ]:
import sys
import pathlib
import time

import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [ ]:
# Local
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))

from utils.watertopo import WaterTopo
from utils.simulation import Simulation
from utils.utils import count_parameters, create_sequence
from utils.train import train_and_validate, evaluate_model, train
from models.convlstm import ConvLSTMCell

In [ ]:
#initialize GPU -  In case of windows use cuda instead of nps
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Is CUDA enabled?",torch.cuda.is_available())
print("Number of GPUs",torch.cuda.device_count())

In [ ]:
class BasicCNN(nn.Module):
    def __init__(self, 
                 input_channels=...,
                 output_channels=...,
                 bias=...,
                 kernel_size=(3, 3, 3, 3), 
                 pool_kernel_size=3,
                 pool_stride=1, 
                 nonlinearity=F.relu
                 ):
        super(BasicCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(2, 64, kernel_size=kernel_size[0], padding=1)
        self.conv2 = nn.Conv2d(2, 64, kernel_size=kernel_size[1], padding=1)
        self.conv3 = nn.Conv2d(2, 64, kernel_size=kernel_size[2], padding=1)
        self.conv4 = nn.Conv2d(2, 64, kernel_size=kernel_size[3], padding=1)

        # Pooling has no learnable parameters, so we can just use one
        self.pool = nn.MeanPool2d(kernel_size=pool_kernel_size, stride=pool_stride)

        # Definition of nonlinearity function
        self.nonlinearity=nonlinearity

        # Last layer
        self.fc = nn.Linear(...)

    def forward(self, x):
        # Convolutions
        x = self.pool(self.nonlinearity(self.conv1(x)))
        x = self.pool(self.nonlinearity(self.conv2(x)))
        x = self.pool(self.nonlinearity(self.conv3(x)))
        x = self.pool(self.nonlinearity(self.conv4(x)))

        # Output layer
        x = self.fc(x)

        return x

In [ ]:
model = BasicCNN(input_channels=...,
                 output_channels=...,
                 bias=...,
                 kernel_size=(3, 3, 3, 3), 
                 pool_kernel_size=3,
                 pool_stride=1, 
                 nonlinearity=F.relu
                 )

print(f"CNN --> num. trainable parameters:{count_parameters(model):8d}")

In [ ]:
model, train_losses, best_val_loss, time = train(model,
                                                 sim_amount=100,
                                                 training_size=0.8,
                                                 batch_size=10,
                                                 T=1,
                                                 H=1,
                                                 num_epochs=100,
                                                 lr=0.005,
                                                 criterion=nn.MSELoss(),
                                                 optimizer=optim.AdamW,
                                                 device=device,
                                                 model_name = 'CNN_kevin'
                                                 )